# Yelp API - Lab

## Part I - Set up the DB
Create your SQL queries to create the DB and the appropriate tables

In [2]:
import config
import json 
import requests
import pandas as pd 
import mysql.connector
import time

In [3]:
## Connect to DB server on AWS
cnx = mysql.connector.connect(
    host = config.aws_host,
    user = config.aws_user,
    passwd = config.aws_password,
    database='Yelp_data'
)
print(cnx)

In [4]:
cursor = cnx.cursor()


In [5]:
# Create a table for the Businesses
TABLES = {}
TABLES['businesses'] =  ("""CREATE TABLE businesses 
                        (id varchar (50) not null unique, 
                        name varchar (50) not null, 
                        review_count int(30) not null, 
                        rating float(20) not null,
                        price float (20) not null,
                        PRIMARY KEY (id))
                        engine=innoDB ;
                        """)

TABLES['reviews'] = ("""CREATE TABLE Reviews
                    (r_id varchar (50) not null,
                    review_text longblob not null,
                    time_created datetime not null,
                    PRIMARY KEY(r_id),
                    FOREIGN KEY (id) REFERNCES (businesses.id)
                    )engine=innoDB;""")
# cursor.execute(create_businesses)

In [13]:
#select business IDs from businesses table 

cursor.execute('''select id from businesses''')
result = cursor.fetchall()
#result = {n:x[0] for (n, x) in enumerate(result)}
result = [x[0] for x in result]
result

['1fVp9c5Vw04ig171a_juRw',
 '9gktK6DNHb3k0y-o6nP7pA',
 'BAKjtydOJj6KrsGCo-6DEQ',
 'CKG7LVmSlvI93scyb1aVeg',
 'dRKdRpPEo6rJ2X0RYbfZzQ',
 'FEPObR6IYWOFapJDwFhGTA',
 'hUdRX0a_2ZZEmB8KCp-WYw',
 'Q1TpfbuPwKPFrNFyu-u2_g',
 'W-i6X2KWxRjsG05CgIM9ow',
 'xH7mEjh4Rn6KdHjXYNoOjw']

## Part 2: Create ETL pipeline for the business data from the API

In [12]:
# Write a function to make a call to the yelp API
def yelp_API_call_bk_chinese(SEARCH_LIMIT):    
    term = 'Chinese'
    location = 'Brooklyn NY'
    offset = 0

    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(config.api_key)}
    url_params = {'term':term.replace(' ','+'),'location':location.replace(' ','+'),'limit':50, 'offset':offset}

    response = requests.get(url, headers=headers, params=url_params)
    
    results = json.loads(response.content)
    
    return results



In [6]:
businesses_columns = ['id','name','review_count','rating','price']

In [14]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB
## I want to return a list of tuples. 
## the tuples contain the values from each dict in the 'businesses' key of the API call function results 
def parse_business_results(api_call):
    df = pd.DataFrame(api_call['businesses'])
    df = df[businesses_columns]
    df['price'] = df['price'].map(lambda x: len(x),na_action='ignore')
    df['price'].fillna(0,inplace=True)
    #df['coordinates']=df['coordinates'].map(lambda x: tuple(x.values()))
    return [tuple(x) for x in df.values]

In [ ]:
def big_business():
    num = 

## yelp api call for reviews 

In [90]:
def businessid_query():
    cursor.execute('''select id from businesses''')
    result = cursor.fetchall()
    result = [x[0] for x in result]
    return result
businessid_query()

['_XLLSXPIjgoZQtwU3RCcQA',
 'lgHmMRBaU-48kGaNz3jYxg',
 'tDYk6DmmU3buWBs9G0NBXQ',
 'U2aUFZARLeSMf4RQIxjuTw',
 'YfJZBmm9tfW--W_WIGReqA']

In [21]:
def yelp_api_call_reviews(r_ids):
    outputs = []
    for r_id in r_ids:
        url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(r_id)
        headers = {'Authorization': 'Bearer {}'.format(config.api_key)}

        response = requests.get(url, headers=headers)
    
        outputs.append(response.content)
        
        outputs = [json.loads(x) for x in outputs]
        
        dfs = []

        for item in outputs:
            dfs.append(pd.DataFrame(item['reviews']))
            df = pd.concat(dfs)
            df = df[['id','text','time_created']]
            df['business_id'] = r_id
            return [tuple(x) for x in df.values]

            #return df.head()

## draft a function to parse yelp reviews results 

In [111]:
reviews_test = yelp_api_call_reviews(businessid_query())

In [136]:
# def parse_review_results(api_call):
#     df = pd.DataFrame(api_call['businesses'])
#     df = df[businesses_columns]
#     df['price'] = df['price'].map(lambda x: len(x),na_action='ignore')
#     df['price'].fillnda(0,inplace=True)
#     #df['coordinates']=df['coordinates'].map(lambda x: tuple(x.values()))
#     


## inserting info to Database

In [11]:
# Write a function to take your parsed data and insert it into the DB
business_stmt = """INSERT INTO businesses (id, name, review_count, rating, price) VALUES (%s, %s, %s, %s, %s)"""

In [12]:
def business_agg(SEARCH_LIMIT):
    cursor.executemany(business_stmt,parse_business_results(yelp_API_call_bk_chinese(SEARCH_LIMIT)))
    cnx.commit()
    return cursor.execute("""SELECT * FROM businesses;""")

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews


In [ ]:
# Write a function to parse out the relevant information from the reviews

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 4: Write SQL queries that will answer the questions posed. 

In [ ]:
# create connection

In [ ]:
# execute SQL queries

## sample code to help with pagination 

In [ ]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
def parse_results(results):
    
    return parsed_results

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time



def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()